In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Reasoning Engine in Vertex AI - Chat history management
@forusone (shins777@gmail.com)

This colab explains how to deal with chat history being stored in firestore.   

Reference : https://python.langchain.com/v0.2/docs/integrations/memory/google_firestore/


## Init Vertex AI SDK

In [5]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    "google-cloud-firestore" \
    "langchain-google-firestore" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    requests

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Configuration and initialization

In [3]:
# Static configuration
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://reasoning_7424"
MODEL = "gemini-1.5-flash"

# Firestore
DATABASE="chat-history"
COLLECTION="history"


### Initialize Vertex AI

In [ ]:
import vertexai
from vertexai.preview import reasoning_engines

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

## Firestore setting for chat history

### Helper method to get session history

In [15]:

# refer to : https://python.langchain.com/v0.2/docs/integrations/memory/google_firestore/#custom-client

def get_session_history(session_id: str):
    """
    Retrieve chat history data with session id for each conversation from Firestore on GCP
    """

    from langchain_google_firestore import FirestoreChatMessageHistory
    from google.cloud import firestore

    client = firestore.Client(project=PROJECT_ID, database=DATABASE)

    return FirestoreChatMessageHistory(
        client=client,
        session_id=session_id,
        collection=COLLECTION,
        encode_message=False,
    )

### Unit test of Firestore

In [24]:
session_id = "conv-003"

chat_history = get_session_history(session_id)

chat_history.add_user_message("Hi! Agent!!")
chat_history.add_ai_message("How can I help you?")

chat_history.add_user_message("I would like to know how to build python code to make a game.")
chat_history.add_ai_message("I got it, It is easy if you can use Generative AI, so you are just needed to think of scenario of the game.")


In [25]:
print(chat_history.messages)

[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}), AIMessage(content='How can I help you?', additional_kwargs={}, response_metadata={}), HumanMessage(content='I would like to know how to build python code to make a game.', additional_kwargs={}, response_metadata={}), AIMessage(content='I got it, It is easy if you can use Generative AI, so you are just needed to think of scenario of the game.', additional_kwargs={}, response_metadata={})]


In [26]:
chat_history.clear()
print(chat_history.messages)

[]


## Integrate reasoning engine and chat history

### Tools definition

In [28]:
# Define Google Search tool
from vertexai.generative_models import grounding, Tool
grounded_search_tool = Tool.from_google_search_retrieval(
    grounding.GoogleSearchRetrieval()
)

### Agent - Local unit test

In [29]:
from vertexai.preview.generative_models import ToolConfig

# Agent Define
agent = reasoning_engines.LangchainAgent(
    model=MODEL,
    tools=[grounded_search_tool,
    ],
    agent_executor_kwargs={"return_intermediate_steps": False},
)

### Add chat come from interaction into chat history


In [34]:
# Agent local unit test before deploying.
respone = agent.query(input="What are the latest main topics in Korean society in term of economy. describe the date specifically.")

In [35]:
chat_history.add_user_message(respone['input'])
chat_history.add_ai_message(respone['output'])

In [36]:
# Agent local unit test before deploying.
respone = agent.query(input="What's the exchange rate from US dollars to KRW currency today?")

In [37]:
chat_history.add_user_message(respone['input'])
chat_history.add_ai_message(respone['output'])

In [38]:
print(chat_history.messages)

[HumanMessage(content='What are the latest main topics in Korean society in term of economy. describe the date specifically.', additional_kwargs={}, response_metadata={}), AIMessage(content="The latest main topics in Korean society in terms of economy, specifically as of July 2024, are:\n\n* **Strong economic recovery:** Supported by robust exports, Korea's GDP growth has rebounded.\n* **Stable employment:** Employment levels remain high, and unemployment rates are low.\n* **Rising interest rates and stabilized housing prices:** Interest rates have likely peaked, and housing prices have stabilized. These factors are expected to support consumption growth.\n* **High household debt:**  A concerning issue is the high level of household debt, which could pose risks to financial stability.\n* **Project finance concerns:** Construction-related project finance is becoming a cause for financial stability concerns.\n* **Low fertility rate:** The lowest fertility rate globally is putting pressur